In [2]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser

system_prompt = "Think through your response step by step. {helper_response}"
model_name = "llama3-8b-8192"
llm_kwargs = {}

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder(variable_name="messages", optional=True),
    ("human", "{input}")
])

assert 'helper_response' in prompt.input_variables
llm = ChatGroq(model=model_name, **llm_kwargs)

chain = prompt | llm | StrOutputParser()
chain

ChatPromptTemplate(input_variables=['helper_response', 'input'], optional_variables=['messages'], input_types={'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'messages': []}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['helper_response'], template='Think through your response step by step. {helper_response}')), MessagesPlaceholder(variable_name='messages', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7eb40903cbd0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7eb40a1de510>, model_name='llama3-8b-8192', groq_api_k

In [8]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableSerializable
# from langchain_core.output_parsers import StrOutputParser

from moa.agent.prompts import SYSTEM_PROMPT, REFERENCE_SYSTEM_PROMPT
from moa.agent.moa import MOAgent
# if not layer_agent_config:
layer_agent_config = {
    'layer_agent_1' : {'system_prompt': SYSTEM_PROMPT, 'model_name': 'llama3-8b-8192'},
    'layer_agent_2' : {'system_prompt': SYSTEM_PROMPT, 'model_name': 'gemma-7b-it'},
    'layer_agent_3' : {'system_prompt': SYSTEM_PROMPT, 'model_name': 'mixtral-8x7b-32768'}
}

parallel_chain_map = dict()
for key, value in layer_agent_config.items():
    chain = MOAgent._create_agent_from_system_prompt(
        system_prompt=value.pop("system_prompt", SYSTEM_PROMPT), 
        model_name=value.pop("model_name", 'llama3-8b-8192'),
        **value
    )
    parallel_chain_map[key] = RunnablePassthrough() | chain

    # chain = parallel_chain_map | RunnableLambda(MOAgent.concat_response)
    # chain
    print(chain)

first=ChatPromptTemplate(input_variables=['helper_response', 'input'], optional_variables=['messages'], input_types={'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'messages': []}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['helper_response'], template='You are a personal assistant that is helpful.\n\n{helper_response}')), MessagesPlaceholder(variable_name='messages', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))]) middle=[ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7eb408fcdb90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7eb408fcf510>, model_name='llama3-8

In [7]:
print(parallel_chain_map)

{'layer_agent_1': RunnablePassthrough()
| ChatPromptTemplate(input_variables=['helper_response', 'input'], optional_variables=['messages'], input_types={'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'messages': []}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['helper_response'], template='You are a personal assistant that is helpful.\n\n{helper_response}')), MessagesPlaceholder(variable_name='messages', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7eb40a514310>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7eb40

In [5]:
print(chain)

first={
  layer_agent_1: RunnablePassthrough()
                 | ChatPromptTemplate(input_variables=['helper_response', 'input'], optional_variables=['messages'], input_types={'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'messages': []}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['helper_response'], template='You are a personal assistant that is helpful.\n\n{helper_response}')), MessagesPlaceholder(variable_name='messages', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])
                 | ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7eb40904e450>, async_client=<groq.resources.chat.compl